# Expression Data Streaming Example

## Set instance and token

In [3]:
suppressMessages(library(tidyverse))
suppressMessages(library(data.table))
suppressMessages(library(httr))
suppressMessages(library(jsonlite))
suppressMessages(library(integrationCurator)) # Genestack client library

# Change the following settings according to your instance
token = '<your token>' 
host = '<your instance>'
version = 'default-released'
base_url = 'frontend/rs/genestack' 

Sys.setenv(PRED_SPOT_HOST=host,
           PRED_SPOT_TOKEN=token,
           PRED_SPOT_VERSION=version)

## Get samples

In [4]:
study_id = 'GSF1061557'
study_filter = paste0('genestack:accession=', study_id)
sample_filter = paste(sprintf('"Cell Type"="%s"',
                              c('CD4-positive, alpha-beta T cell', 'CD8-positive, alpha-beta T cell')),collapse=" OR ")

start = Sys.time()
samples <- as_tibble(integrationCurator::OmicsQueriesApi_search_samples(
    study_filter=study_filter,
    sample_filter=sample_filter
)$content$data[['metadata']])
cat(sprintf('Time to get %s samples: %s seconds\n\n', nrow(samples), round(Sys.time()-start)))

head(samples)

Time to get 40 samples: 0 seconds



genestack:accession,Sample Source ID,Sample Name,Organism,Sex,Disease,Age,Age Unit,Tissue,Cell Type,...,Cells,Sample Source,Type,Replicate,groupId,DMARDs at Draw,Has lupus nephritis?,ANA,Prednisone Dose,dsDNA
GSF1061595,P4415_CD4_1,NA,Homo sapiens,female,Healthy,59,NA,blood,"CD4-positive, alpha-beta T cell",...,CD4+,GSF1061557,HC,1,GSF1061558,NA,NA,NA,NA,NA
GSF1061597,P4415_CD8_1,NA,Homo sapiens,female,Healthy,59,NA,blood,"CD8-positive, alpha-beta T cell",...,CD8+,GSF1061557,HC,1,GSF1061558,NA,NA,NA,NA,NA
GSF1061596,P4415_CD4_2,NA,Homo sapiens,female,Healthy,59,NA,blood,"CD4-positive, alpha-beta T cell",...,CD4+,GSF1061557,HC,2,GSF1061558,NA,NA,NA,NA,NA
GSF1061590,P3449_CD8_2,NA,Homo sapiens,female,Healthy,81,NA,blood,"CD8-positive, alpha-beta T cell",...,CD8+,GSF1061557,HC,2,GSF1061558,NA,NA,NA,NA,NA
GSF1061603,P6375_CD4_1,NA,Homo sapiens,female,systemic lupus erythematosus,59,NA,blood,"CD4-positive, alpha-beta T cell",...,CD4+,GSF1061557,SLE,1,GSF1061558,"hydroxychloroquine, prednisone",No,1/2560,15 mg,3
GSF1061605,P6375_CD8_1,NA,Homo sapiens,female,systemic lupus erythematosus,59,NA,blood,"CD8-positive, alpha-beta T cell",...,CD8+,GSF1061557,SLE,1,GSF1061558,"hydroxychloroquine, prednisone",No,1/2560,15 mg,3


## Get expression data

In [6]:
# Extract expression data from omics/expression/streamed-data
start = Sys.time()

expression_groups = as_tibble(ExpressionIntegrationApi_get_parents_by_study(id=study_id)$content)

# there can be multiple expression groups linked to the same samples
# this example takes the the first one
group_accession = expression_groups[1, 'itemId']
                     
# Get expression data: return a 2-row csv
streamed_expressions = httr::GET(sprintf('https://%s/%s/integrationCurator/%s/omics/expression/streamed-data', host, base_url, version),
    add_headers(accept = "gzip", `Genestack-API-Token` = token), 
    query = list(
        groupAccession = group_accession,
        sampleFilter = sample_filter
    ))$content               
 
end = Sys.time()

cat(sprintf('Time to extract expression values from the endpoint: %s seconds\n', round(end-start, digit = 1)))           

Time to extract expression values from the endpoint: 1.3 seconds


In [7]:
as_tibble(fread(rawToChar(streamed_expressions), showProgress = FALSE))

NAME,P0772_CD4_1,P0772_CD4_2,P0772_CD8_1,P0772_CD8_2,P1002_CD4_1,P1002_CD4_2,P1002_CD8_1,P1002_CD8_2,P1353_CD4_1,...,P7851_CD8_1,P7851_CD8_2,P7852_CD4_1,P7852_CD4_2,P7852_CD8_1,P7852_CD8_2,P7856_CD4_1,P7856_CD4_2,P7856_CD8_1,P7856_CD8_2
A1BG,189.46,214.08,37.00,15.00,51.91,97.00,10.00,5.00,51.00,...,0.00,0.00,200.00,7.00,14.00,62.00,58.00,10.00,76.00,15.00
A1BG-AS1,0.25,17.11,0.00,1.21,0.00,9.53,5.46,0.00,4.15,...,0.00,0.00,0.20,0.00,0.00,0.00,0.00,0.00,1.06,0.79
A1CF,1.04,5.97,8.50,2.95,1.00,0.00,10.76,15.74,1.12,...,0.00,0.00,1.37,3.17,9.22,3.00,3.00,0.00,5.54,1.00
A2M,0.00,0.00,0.00,0.00,6.00,0.00,1.00,0.00,0.00,...,0.00,0.00,5.00,5.00,3.75,4.00,0.00,0.00,13.46,2.00
A2M-AS1,0.00,0.00,155.00,143.00,0.00,0.00,0.00,13.72,0.00,...,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,11.65,0.00
A2ML1,8.33,15.82,28.55,17.33,27.90,12.24,80.07,113.93,29.59,...,0.00,1.91,1.37,5.92,4.95,12.89,11.12,1.15,8.75,11.44
A2MP1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
A4GALT,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
A4GNT,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
AA06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
